In [1]:
import os
import pandas as pd
import numpy as np
import random
import gc
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
from gensim.corpora import WikiCorpus
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from sklearn.feature_extraction.text import TfidfVectorizer
from  collections import Counter

np.random.seed(2019)
random.seed(2019)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 280)
pd.set_option('display.max_colwidth', 150)
data_path = '/data/workspace/kimi/tencent_ads/2020/dataset'
preprocess_path = 'preprocess'

In [3]:
clk_df = pd.read_csv(f'{data_path}/train_preliminary/click_log.csv' )
clk_test_df = pd.read_csv(f'{data_path}/test/click_log.csv' )
clk_df = pd.concat([clk_df,clk_test_df],axis=0)
print(clk_df)


          time  user_id  creative_id  click_times
0            9    30920       567330            1
1           65    30920      3072255            1
2           56    30920      2361327            1
3            6   309204       325532            1
4           59   309204      2746730            1
...        ...      ...          ...          ...
33585507    16  3131989       585914            1
33585508    13  3131989       749134            1
33585509    12  3131989       589027            1
33585510    61  3131989      2404831            1
33585511    14  3131989       665822            1

[63668283 rows x 4 columns]


In [ ]:
adv_count_value= clk_df['user_id'].value_counts()
print(len(adv_count_value))
print(adv_count_value[:10])
print(adv_count_value[-10000:-9000])
print(adv_count_value[-50000:-40000])
print(adv_count_value[-500000:-400000])
print(adv_count_value[-1500000:-1400000])
#adv_count_value = adv_count_value[adv_count_value.values < 200]

#null_counts = adv_count_value['\\N']
#print(f"null:{null_counts}")
x = adv_count_value.index
y = adv_count_value.values
arr = np.array(adv_count_value.values)
print(len(arr))
print(np.mean(arr))
print(np.median(arr))
print(np.std(arr))

plt.scatter(x,y)
plt.show()


In [ ]:
adv_count_value= clk_df['time'].value_counts()
print(len(adv_count_value))
print(adv_count_value[:10])
print(adv_count_value)
#null_counts = adv_count_value['\\N']
#print(f"null:{null_counts}")
x = adv_count_value.index
y = adv_count_value.values
plt.scatter(x,y)
plt.show()

In [ ]:
user_click_sum_df = clk_df.groupby(['user_id']).click_times.sum().reset_index().rename(columns={'click_times':'click_times_total'}).sort_values(by='click_times_total')
print(user_click_sum_df)
drop_user_id_df = user_click_sum_df[user_click_sum_df.click_times_total > 900]
print(drop_user_id_df)
user_click_sum_df =  user_click_sum_df[user_click_sum_df.click_times_total <= 900]
print(user_click_sum_df)
x = user_click_sum_df['user_id'].values.tolist()
y = user_click_sum_df['click_times_total'].values.tolist()
plt.scatter(x,y)
plt.show()

In [ ]:
day_uv_df = clk_df.groupby(['time']).user_id.nunique().reset_index().rename(columns={'user_id':'day_uv'}).sort_values(by='day_uv')
print(day_uv_df)
x = day_uv_df['time'].values.tolist()
y = day_uv_df['day_uv'].values.tolist()
plt.scatter(x,y)
plt.show()

In [ ]:
user_p_df  = clk_df[clk_df.user_id == 157585].groupby(['time']).click_times.count().reset_index().rename(columns={'click_times':'creative_id'})
print(user_p_df)
x = user_p_df['time'].values.tolist()
y = user_p_df['creative_id'].values.tolist()
plt.scatter(x,y)
plt.show()

In [ ]:
user_p_df  = clk_df[clk_df.user_id == 711447].groupby(['time']).click_times.count().reset_index().rename(columns={'click_times':'creative_id'})
pd.set_option('display.max_rows', 100)
print(user_p_df)
pd.set_option('display.max_rows', 10)
x = user_p_df['time'].values.tolist()
y = user_p_df['creative_id'].values.tolist()
plt.scatter(x,y)
plt.show()

In [ ]:
user_ps_df  = clk_df[clk_df.user_id == 727101].groupby(['time']).click_times.sum().reset_index().rename(columns={'click_times':'creative_id'})
print(user_ps_df)
x = user_ps_df['time'].values.tolist()
y = user_ps_df['creative_id'].values.tolist()
plt.scatter(x,y)
plt.show()

In [ ]:
ad_df = pd.read_csv(f'{data_path}/train_preliminary/ad.csv' )

In [ ]:
user_ins_df = clk_df[clk_df.user_id == 157585]
user_ins_df= user_ins_df.merge(ad_df,on='creative_id').sort_values(by='time')
print(user_ins_df)

In [ ]:
user_ins_df = clk_df[clk_df.user_id == 727101]
user_ins_df= user_ins_df.merge(ad_df,on='creative_id').sort_values(by='time')
print(user_ins_df)

In [ ]:
user_ins_df = clk_df[clk_df.user_id == 420997]
user_ins_df= user_ins_df.merge(ad_df,on='creative_id').sort_values(by='time')
print(user_ins_df)

In [ ]:
user_ins_df = clk_df[clk_df.user_id == 737997]
user_ins_df= user_ins_df.merge(ad_df,on='creative_id').sort_values(by='time')
pd.set_option('display.max_rows', 100)
print(user_ins_df)
pd.set_option('display.max_rows', 10)

In [ ]:
user_ins_df = clk_df[clk_df.user_id == 867303]
user_ins_df= user_ins_df.merge(ad_df,on='creative_id').sort_values(by='time')
pd.set_option('display.max_rows', 100)
print(user_ins_df)
pd.set_option('display.max_rows', 10)

In [ ]:
user_ins_df = clk_df[clk_df.user_id == 323781]
user_ins_df= user_ins_df.merge(ad_df,on='creative_id').sort_values(by='time')
pd.set_option('display.max_rows', 100)
print(user_ins_df)
pd.set_option('display.max_rows', 10)

In [ ]:
merged_df = clk_df.merge(ad_df,on='creative_id')
merged_df.replace("\\N",-1,inplace=True)
merged_df = merged_df.fillna(-1).astype(float)
merged_df['wday'] = merged_df['time'].apply(lambda x :int(x /7))
merged_df['wday1'] = merged_df['time'].apply(lambda x :int(x %7))
merged_df['month'] = merged_df['time'].apply(lambda x :int(x /30))
print(merged_df)
del clk_df
gc.collect()

In [ ]:
adv_count_value= merged_df['ad_id'].value_counts()
print(len(adv_count_value))
print(adv_count_value[:10])
null_counts = adv_count_value.get(-1, 0)
print(f"null:{null_counts}")
# x = [str(i) for i in adv_count_value.index]
# y = adv_count_value.values
# plt.scatter(x,y)
# plt.show()

In [ ]:
adv_count_value= merged_df['creative_id'].value_counts()
print(len(adv_count_value))
print(adv_count_value[:10])
null_counts = adv_count_value.get(-1, 0)
print(f"null:{null_counts}")
# x = [str(i) for i in adv_count_value.index]
# y = adv_count_value.values
# plt.scatter(x,y)
# plt.show()

In [ ]:
print(merged_df.groupby('wday1').creative_id.count())
pd.set_option('display.max_rows', 100)
print(merged_df[merged_df.wday1 == 0].groupby('time').creative_id.count())
print(merged_df[merged_df.wday1 == 1].groupby('time').creative_id.count())
print(merged_df[merged_df.wday1 == 2].groupby('time').creative_id.count())
pd.set_option('display.max_rows', 10)

In [ ]:
filter_user_df  = merged_df.merge(user_click_sum_df,on='user_id')
filter_user_df = filter_user_df.fillna(-1)
filter_user_df = filter_user_df[filter_user_df.click_times_total > 0]

print(filter_user_df)



In [ ]:
print(filter_user_df.groupby('wday1').creative_id.count())
pd.set_option('display.max_rows', 100)
print(filter_user_df[filter_user_df.wday1 == 0].groupby('time').creative_id.count())
print(filter_user_df[filter_user_df.wday1 == 1].groupby('time').creative_id.count())
print(filter_user_df[filter_user_df.wday1 == 2].groupby('time').creative_id.count())
pd.set_option('display.max_rows', 10)

del filter_user_df
gc.collect() 

In [ ]:
adv_count_value= merged_df['product_category'].value_counts()
print(len(adv_count_value))
print(adv_count_value[:10])
null_counts = adv_count_value.get(-1, 0)
print(f"null:{null_counts}")
x = adv_count_value.index
y = adv_count_value.values
plt.scatter(x,y)
plt.show()

In [ ]:
adv_count_value= merged_df['industry'].value_counts()
print(len(adv_count_value))
print(adv_count_value[:10])
null_counts = adv_count_value.get(-1, 0)
print(f"null:{null_counts}")
x = [str(i) for i in adv_count_value.index]
y = adv_count_value.values
plt.scatter(x,y)
plt.show()

In [ ]:
adv_count_value= merged_df['advertiser_id'].value_counts()
print(len(adv_count_value))
print(adv_count_value[:10])
null_counts = adv_count_value.get(-1, 0)
print(f"null:{null_counts}")
# x = [str(i) for i in adv_count_value.index]
# y = adv_count_value.values
# plt.scatter(x,y)
# plt.show()

In [ ]:
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==14681 ].groupby(['time']).creative_id.count()
print(advertiser_id_spec_df)
x = [str(i) for i in advertiser_id_spec_df.index]
y = advertiser_id_spec_df.values
plt.scatter(x,y)
plt.show()

In [ ]:
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==10989 ].groupby(['time']).creative_id.count()
print(advertiser_id_spec_df)
x = [str(i) for i in advertiser_id_spec_df.index]
y = advertiser_id_spec_df.values
plt.scatter(x,y)
plt.show()

In [ ]:
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==14682 ].groupby(['time']).creative_id.count()
print(advertiser_id_spec_df)
x = [str(i) for i in advertiser_id_spec_df.index]
y = advertiser_id_spec_df.values
plt.scatter(x,y)
plt.show()

In [ ]:
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==22885 ].groupby(['time']).creative_id.count()
print(advertiser_id_spec_df)
x = [str(i) for i in advertiser_id_spec_df.index]
y = advertiser_id_spec_df.values
plt.scatter(x,y)
plt.show()

In [ ]:
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==14681 ].groupby(['ad_id']).creative_id.count()
print(advertiser_id_spec_df)
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==14681 ].groupby(['product_id']).creative_id.count()
print(advertiser_id_spec_df)
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==14681 ].groupby(['industry']).creative_id.count()
print(advertiser_id_spec_df)


In [ ]:
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==22885 ].groupby(['ad_id']).creative_id.count()
print(advertiser_id_spec_df)
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==22885 ].groupby(['product_id']).creative_id.count()
print(advertiser_id_spec_df)
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==22885 ].groupby(['industry']).creative_id.count()
print(advertiser_id_spec_df)

In [ ]:
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==14682 ].groupby(['ad_id']).creative_id.count()
print(advertiser_id_spec_df)
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==14682 ].groupby(['product_id']).creative_id.count()
print(advertiser_id_spec_df)
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==14682 ].groupby(['industry']).creative_id.count().sort_values()
print(advertiser_id_spec_df)

In [ ]:
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==918 ].groupby(['ad_id']).creative_id.count()
print(advertiser_id_spec_df)
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==918 ].groupby(['product_id']).creative_id.count()
print(advertiser_id_spec_df)
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==918 ].groupby(['industry']).creative_id.count().sort_values()
print(advertiser_id_spec_df)

In [ ]:

advertiser_id_spec_df = merged_df[merged_df.advertiser_id==9058 ].groupby(['ad_id']).creative_id.count()
print(advertiser_id_spec_df)
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==9058 ].groupby(['product_id']).creative_id.count().sort_values()
print(advertiser_id_spec_df)
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==9058 ].groupby(['industry']).creative_id.count().sort_values()
print(advertiser_id_spec_df)

In [ ]:
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==10986 ].groupby(['ad_id']).creative_id.count()
print(advertiser_id_spec_df)
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==10986 ].groupby(['product_id']).creative_id.count().sort_values()
print(advertiser_id_spec_df)
advertiser_id_spec_df = merged_df[merged_df.advertiser_id==10986 ].groupby(['industry']).creative_id.count().sort_values()
print(advertiser_id_spec_df)

In [ ]:
industry_spec_df = merged_df[merged_df.industry==6].groupby(['advertiser_id']).creative_id.count().sort_values()
print(industry_spec_df)
industry_spec_df = merged_df[merged_df.industry==6 ].groupby(['product_id']).creative_id.count().sort_values()
print(industry_spec_df)
#print(advertiser_id_spec_df)
# advertiser_id_spec_df = merged_df[merged_df.advertiser_id==10986 ].groupby(['product_id']).creative_id.count().sort_values()
# print(advertiser_id_spec_df)
# advertiser_id_spec_df = merged_df[merged_df.advertiser_id==10986 ].groupby(['industry']).creative_id.count().sort_values()
# print(advertiser_id_spec_df)

In [ ]:
industry_spec_df = merged_df[merged_df.industry==319].groupby(['advertiser_id']).creative_id.count().sort_values()
print(industry_spec_df)
industry_spec_df = merged_df[merged_df.industry==319 ].groupby(['product_id']).creative_id.count().sort_values()
print(industry_spec_df)

In [ ]:
adv_count_value= merged_df['product_id'].value_counts()
print(len(adv_count_value))
print(adv_count_value[:10])
null_counts = adv_count_value.get(-1, 0)
print(f"null:{null_counts}")
# x = [str(i) for i in adv_count_value.index]
# y = adv_count_value.values
# plt.scatter(x,y)
# plt.show()

In [ ]:
advertiser_id_spec_df = merged_df[merged_df.product_id==129 ].groupby(['ad_id']).creative_id.count()
print(advertiser_id_spec_df)
advertiser_id_spec_df = merged_df[merged_df.product_id==129 ].groupby(['advertiser_id']).creative_id.count().sort_values()
print(advertiser_id_spec_df)
advertiser_id_spec_df = merged_df[merged_df.product_id==129 ].groupby(['industry']).creative_id.count().sort_values()
print(advertiser_id_spec_df)

In [ ]:
advertiser_id_spec_df = merged_df[merged_df.product_id==1261 ].groupby(['ad_id']).creative_id.count()
print(advertiser_id_spec_df)
advertiser_id_spec_df = merged_df[merged_df.product_id==1261].groupby(['advertiser_id']).creative_id.count().sort_values()
print(advertiser_id_spec_df)
advertiser_id_spec_df = merged_df[merged_df.product_id==1261 ].groupby(['industry']).creative_id.count().sort_values()
print(advertiser_id_spec_df)